In [1]:
import tkinter as tk
from tkinter import messagebox
import copy
import math

# --- Move Representation ---
class Move:
    def __init__(self, src, dst, piece, capture=False, promotion=None, en_passant=False, castling=False):
        self.src = src
        self.dst = dst
        self.piece = piece
        self.capture = capture
        self.promotion = promotion
        self.en_passant = en_passant
        self.castling = castling

# --- Piece Base Class ---
class Piece:
    def __init__(self, color, pos):
        self.color = color
        self.pos = pos
        self.name = ''
    def get_valid_moves(self, board):
        return []

class King(Piece):
    def __init__(self, color, pos):
        super().__init__(color, pos)
        self.name = 'K'
        self.has_moved = False
    def get_valid_moves(self, board):
        moves = []
        r, c = self.pos
        directions = [(-1,-1),(-1,0),(-1,1),(0,-1),(0,1),(1,-1),(1,0),(1,1)]
        for dr, dc in directions:
            nr, nc = r+dr, c+dc
            if board.on_board(nr, nc) and (not board.is_occupied(nr,nc) or board.grid[nr][nc].color!=self.color):
                moves.append(Move((r,c),(nr,nc),self, capture=board.is_occupied(nr,nc)))
        if not self.has_moved and not board.in_check(self.color):
            for side in ['king','queen']:
                if board.can_castle(self.color, side):
                    dc = 2 if side=='king' else -2
                    moves.append(Move((r,c),(r,c+dc),self, castling=True))
        return moves

class Queen(Piece):
    def __init__(self, color, pos):
        super().__init__(color, pos)
        self.name = 'Q'
    def get_valid_moves(self, board):
        return board.ray_moves(self, [(1,0),(-1,0),(0,1),(0,-1),(1,1),(1,-1),(-1,1),(-1,-1)])

class Rook(Piece):
    def __init__(self, color, pos):
        super().__init__(color, pos)
        self.name = 'R'
        self.has_moved = False
    def get_valid_moves(self, board):
        return board.ray_moves(self, [(1,0),(-1,0),(0,1),(0,-1)])

class Bishop(Piece):
    def __init__(self, color, pos):
        super().__init__(color, pos)
        self.name = 'B'
    def get_valid_moves(self, board):
        return board.ray_moves(self, [(1,1),(1,-1),(-1,1),(-1,-1)])

class Knight(Piece):
    def __init__(self, color, pos):
        super().__init__(color, pos)
        self.name = 'N'
    def get_valid_moves(self, board):
        moves = []
        r, c = self.pos
        for dr, dc in [(2,1),(2,-1),(-2,1),(-2,-1),(1,2),(1,-2),(-1,2),(-1,-2)]:
            nr, nc = r+dr, c+dc
            if board.on_board(nr, nc) and (not board.is_occupied(nr,nc) or board.grid[nr][nc].color!=self.color):
                moves.append(Move((r,c),(nr,nc),self, capture=board.is_occupied(nr,nc)))
        return moves

class Pawn(Piece):
    def __init__(self, color, pos):
        super().__init__(color, pos)
        self.name = 'P'
    def get_valid_moves(self, board):
        moves = []
        r, c = self.pos
        d = -1 if self.color=='white' else 1
        if board.on_board(r+d, c) and not board.is_occupied(r+d, c):
            if (r+d == 0 and self.color == 'white') or (r+d == 7 and self.color == 'black'):
                for promo in ['Q','R','B','N']:
                    moves.append(Move((r,c),(r+d,c), self, promotion=promo))
            else:
                moves.append(Move((r,c),(r+d,c), self))
            start_row = 6 if self.color == 'white' else 1
            if r == start_row and not board.is_occupied(r + 2*d, c):
                moves.append(Move((r,c),(r+2*d,c), self))
        for dc in (-1, 1):
            nr, nc = r + d, c + dc
            if board.on_board(nr, nc):
                target = board.grid[nr][nc]
                if target and target.color != self.color:
                    if (nr == 0 and self.color == 'white') or (nr == 7 and self.color == 'black'):
                        for promo in ['Q','R','B','N']:
                            moves.append(Move((r,c),(nr,nc), self, capture=True, promotion=promo))
                    else:
                        moves.append(Move((r,c),(nr,nc), self, capture=True))
        ep = board.en_passant_target
        if ep and r == (3 if self.color == 'white' else 4):
            if (ep[0] == r and abs(ep[1] - c) == 1):
                moves.append(Move((r,c),(r+d, ep[1]), self, capture=True, en_passant=True))
        return moves

class Board:
    def __init__(self):
        self.grid = [[None]*8 for _ in range(8)]
        self.en_passant_target = None
        self.setup()
    def setup(self):
        for c in range(8):
            self.grid[6][c] = Pawn('white', (6,c))
            self.grid[1][c] = Pawn('black', (1,c))
        order = [Rook, Knight, Bishop, Queen, King, Bishop, Knight, Rook]
        for c, cls in enumerate(order):
            self.grid[7][c] = cls('white', (7,c))
            self.grid[0][c] = cls('black', (0,c))
    def on_board(self, r, c): return 0<=r<8 and 0<=c<8
    def is_occupied(self, r, c): return self.grid[r][c] is not None
    def move_piece(self, m):
        sr, sc = m.src; dr, dc = m.dst; p = self.grid[sr][sc]
        if m.castling:
            if dc > sc:  # Kingside
                rook = self.grid[sr][7]
                self.grid[sr][5] = rook
                rook.pos = (sr,5)
                self.grid[sr][7] = None
                rook.has_moved = True
            else:  # Queenside
                rook = self.grid[sr][0]
                self.grid[sr][3] = rook
                rook.pos = (sr,3)
                self.grid[sr][0] = None
                rook.has_moved = True
            p.has_moved = True
        if m.en_passant:
            self.grid[sr][dc] = None
        if m.promotion:
            p = {'Q':Queen, 'R':Rook, 'B':Bishop, 'N':Knight}[m.promotion](p.color, (dr,dc))
        self.grid[dr][dc] = p
        p.pos = (dr, dc)
        self.grid[sr][sc] = None
        self.en_passant_target = None
        if isinstance(p, Pawn) and abs(dr - sr) == 2:
            self.en_passant_target = ((sr + dr) // 2, sc)
        if hasattr(p, 'has_moved'):
            p.has_moved = True
    def all_moves(self, color):
        moves = []
        for r in range(8):
            for c in range(8):
                p = self.grid[r][c]
                if p and p.color == color:
                    for m in p.get_valid_moves(self):
                        b2 = copy.deepcopy(self)
                        b2.move_piece(m)
                        if not b2.in_check(color):
                            moves.append(m)
        return moves
    def in_check(self, color):
        king_pos = None
        for r in range(8):
            for c in range(8):
                p = self.grid[r][c]
                if isinstance(p, King) and p.color == color:
                    king_pos = (r, c)
                    break
            if king_pos:
                break
        if not king_pos:
            return False
        opponent = 'white' if color == 'black' else 'black'
        for r in range(8):
            for c in range(8):
                p = self.grid[r][c]
                if p and p.color == opponent and not isinstance(p, King):
                    for m in p.get_valid_moves(self):
                        if m.dst == king_pos:
                            return True
        return False
    def can_castle(self, color, side):
        row = 7 if color == 'white' else 0
        king = self.grid[row][4]
        if not isinstance(king, King) or king.has_moved:
            return False
        rook_col = 7 if side == 'king' else 0
        rook = self.grid[row][rook_col]
        if not isinstance(rook, Rook) or rook.has_moved:
            return False
        if any(self.grid[row][c] is not None for c in range(min(4, rook_col)+1, max(4, rook_col))):
            return False
        
        # Validate castling path safety
        direction = 1 if side == 'king' else -1
        for step in [1, 2]:
            temp_board = copy.deepcopy(self)
            new_col = 4 + direction * step
            temp_board.move_piece(Move((row,4), (row, new_col), king, castling=True))
            if temp_board.in_check(color):
                return False
        return True
    def ray_moves(self, piece, dirs):
        moves = []
        r, c = piece.pos
        for dr, dc in dirs:
            nr, nc = r + dr, c + dc
            while self.on_board(nr, nc):
                if not self.is_occupied(nr, nc):
                    moves.append(Move((r,c), (nr,nc), piece))
                else:
                    if self.grid[nr][nc].color != piece.color:
                        moves.append(Move((r,c), (nr,nc), piece, capture=True))
                    break
                nr += dr
                nc += dc
        return moves
    def in_checkmate(self, color):
        return self.in_check(color) and not self.all_moves(color)
    def stalemate(self, color):
        return not self.in_check(color) and not self.all_moves(color)

class Evaluation:
    values = {'K': 0, 'Q': 9, 'R': 5, 'B': 3.25, 'N': 3, 'P': 1}
    positional_weights = {
        'P': [
            [0, 0, 0, 0, 0, 0, 0, 0],
            [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5],
            [0.1, 0.1, 0.2, 0.3, 0.3, 0.2, 0.1, 0.1],
            [0.05, 0.05, 0.1, 0.25, 0.25, 0.1, 0.05, 0.05],
            [0, 0, 0, 0.2, 0.2, 0, 0, 0],
            [0.05, -0.05, -0.1, 0, 0, -0.1, -0.05, 0.05],
            [0.05, 0.1, 0.1, -0.2, -0.2, 0.1, 0.1, 0.05],
            [0, 0, 0, 0, 0, 0, 0, 0]
        ],
        'N': [
            [-0.5, -0.4, -0.3, -0.3, -0.3, -0.3, -0.4, -0.5],
            [-0.4, -0.2, 0, 0.05, 0.05, 0, -0.2, -0.4],
            [-0.3, 0.05, 0.1, 0.15, 0.15, 0.1, 0.05, -0.3],
            [-0.3, 0, 0.15, 0.2, 0.2, 0.15, 0, -0.3],
            [-0.3, 0.05, 0.15, 0.2, 0.2, 0.15, 0.05, -0.3],
            [-0.3, 0, 0.1, 0.15, 0.15, 0.1, 0, -0.3],
            [-0.4, -0.2, 0, 0, 0, 0, -0.2, -0.4],
            [-0.5, -0.4, -0.3, -0.3, -0.3, -0.3, -0.4, -0.5]
        ],
        'K': [
            [-0.3, -0.4, -0.4, -0.5, -0.5, -0.4, -0.4, -0.3],
            [-0.3, -0.4, -0.4, -0.5, -0.5, -0.4, -0.4, -0.3],
            [-0.3, -0.4, -0.4, -0.5, -0.5, -0.4, -0.4, -0.3],
            [-0.3, -0.4, -0.4, -0.5, -0.5, -0.4, -0.4, -0.3],
            [-0.2, -0.3, -0.3, -0.4, -0.4, -0.3, -0.3, -0.2],
            [-0.1, -0.2, -0.2, -0.2, -0.2, -0.2, -0.2, -0.1],
            [0.2, 0.2, 0, 0, 0, 0, 0.2, 0.2],
            [0.2, 0.3, 0.1, 0, 0, 0.1, 0.3, 0.2]
        ]
    }

    @staticmethod
    def eval_board(board):
        total = 0
        for r in range(8):
            for c in range(8):
                p = board.grid[r][c]
                if p:
                    value = Evaluation.values[p.name] * (1 if p.color == 'white' else -1)
                    
                    if p.name in Evaluation.positional_weights:
                        table = Evaluation.positional_weights[p.name]
                        adjusted_r = 7 - r if p.color == 'black' else r
                        value += table[adjusted_r][c] * 0.1
                    
                    if p.name == 'K' and not board.in_check(p.color):
                        if (c < 3 or c > 5) and (r < 3 or r > 5):
                            value += 0.1
                        else:
                            value -= 0.2
                    
                    total += value
        return total

class AIPlayer:
    def __init__(self, color, depth=3):
        self.color = color
        self.depth = depth

    def choose_move(self, board):
        def alphabeta(node, depth, alpha, beta, maximizing):
            if depth == 0 or node.in_checkmate(self.color) or node.stalemate(self.color):
                return Evaluation.eval_board(node)
            
            if maximizing:
                max_val = -math.inf
                for move in node.all_moves(self.color):
                    child = copy.deepcopy(node)
                    child.move_piece(move)
                    val = alphabeta(child, depth-1, alpha, beta, False)
                    max_val = max(max_val, val)
                    alpha = max(alpha, val)
                    if beta <= alpha:
                        break
                return max_val
            else:
                min_val = math.inf
                opponent = 'white' if self.color == 'black' else 'black'
                for move in node.all_moves(opponent):
                    child = copy.deepcopy(node)
                    child.move_piece(move)
                    val = alphabeta(child, depth-1, alpha, beta, True)
                    min_val = min(min_val, val)
                    beta = min(beta, val)
                    if beta <= alpha:
                        break
                return min_val
        
        best_move = None
        best_value = -math.inf
        for move in board.all_moves(self.color):
            child = copy.deepcopy(board)
            child.move_piece(move)
            value = alphabeta(child, self.depth-1, -math.inf, math.inf, False)
            if value > best_value or (value == best_value and best_move is None):
                best_value = value
                best_move = move
        return best_move

class HumanPlayer:
    def __init__(self, color):
        self.color = color

class ChessGame:
    def __init__(self):
        self.board = Board()
        self.current = 'white'
        self.human = HumanPlayer('white')
        self.ai = AIPlayer('black', depth=3)
        self.selected_src = None
        self.valid_moves = []
        self.dst_notations = []

        self.cell_size = 60
        self.margin = 20
        size = 8 * self.cell_size + 2 * self.margin

        self.root = tk.Tk()
        self.root.title("Chess vs AI")
        self.canvas = tk.Canvas(self.root, width=size, height=size)
        self.canvas.grid(row=0, column=0, columnspan=2)
        self.entry = tk.Entry(self.root)
        self.entry.grid(row=1, column=0)
        self.btn = tk.Button(self.root, text="Enter", command=self.on_move)
        self.btn.grid(row=1, column=1)
        self.status = tk.Label(self.root, text="")
        self.status.grid(row=2, column=0, columnspan=2)

        self.update_status()
        self.draw_board()
        self.root.mainloop()

    def notation_to_pos(self, notation):
        if len(notation) < 2:
            raise ValueError("Invalid notation")
        col = ord(notation[0].lower()) - ord('a')
        row = 8 - int(notation[1])
        return (row, col)

    def pos_to_notation(self, pos):
        row, col = pos
        return f"{chr(col + ord('a'))}{8 - row}"

    def update_status(self):
        if self.current == self.human.color:
            if not self.selected_src:
                txt = "Human's turn: select piece (e.g. e2)"
            else:
                txt = f"Selected {self.pos_to_notation(self.selected_src)}. Destinations: {', '.join(self.dst_notations)}"
        else:
            txt = "AI's turn: thinking..."
        self.status.config(text=txt)

    def draw_board(self):
        self.canvas.delete("all")
        colors = ["#f0d9b5", "#b58863"]
        for r in range(8):
            for c in range(8):
                x1 = self.margin + c * self.cell_size
                y1 = self.margin + r * self.cell_size
                x2 = x1 + self.cell_size
                y2 = y1 + self.cell_size
                color = colors[(r + c) % 2]
                self.canvas.create_rectangle(x1, y1, x2, y2, fill=color, outline="")
                p = self.board.grid[r][c]
                if p:
                    glyphs = {
                        'K': ('♔', '♚'), 
                        'Q': ('♕', '♛'), 
                        'R': ('♖', '♜'),
                        'B': ('♗', '♝'), 
                        'N': ('♘', '♞'), 
                        'P': ('♙', '♟')
                    }
                    glyph = glyphs[p.name][0] if p.color == 'white' else glyphs[p.name][1]
                    fill_color = 'white' if p.color == 'white' else 'black'
                    self.canvas.create_text((x1 + x2) // 2, (y1 + y2) // 2, 
                                          text=glyph, font=("Arial", 32), fill=fill_color)
        files = 'abcdefgh'
        for i in range(8):
            x = self.margin + i * self.cell_size + self.cell_size // 2
            self.canvas.create_text(x, self.margin//2, text=files[i], font=("Arial",12))
            self.canvas.create_text(x, self.margin + 8*self.cell_size + self.margin//2, 
                                   text=files[i], font=("Arial",12))
        for i in range(8):
            y = self.margin + i * self.cell_size + self.cell_size//2
            self.canvas.create_text(self.margin//2, y, text=str(8-i), font=("Arial",12))
            self.canvas.create_text(self.margin + 8*self.cell_size + self.margin//2, 
                                   y, text=str(8-i), font=("Arial",12))

    def on_move(self):
        txt = self.entry.get().strip().lower()
        self.entry.delete(0, tk.END)
        if self.current == self.human.color:
            if not self.selected_src:
                try:
                    src = self.notation_to_pos(txt)
                except:
                    messagebox.showerror("Error", "Invalid input. Use format like 'e2'.")
                    return
                piece = self.board.grid[src[0]][src[1]]
                if not piece or piece.color != self.human.color:
                    messagebox.showerror("Error", "Select your own piece.")
                    return
                self.selected_src = src
                self.valid_moves = [m for m in self.board.all_moves(self.human.color) if m.src == src]
                if not self.valid_moves:
                    messagebox.showerror("Error", "No valid moves for this piece.")
                    self.selected_src = None
                    return
                self.dst_notations = [self.pos_to_notation(m.dst) + (m.promotion if m.promotion else '') 
                                     for m in self.valid_moves]
                self.update_status()
            else:
                promotion = None
                if len(txt) == 3 and txt[2] in ['q','r','b','n']:
                    promotion = txt[2].upper()
                    dst_txt = txt[:2]
                else:
                    dst_txt = txt
                try:
                    dst = self.notation_to_pos(dst_txt)
                except:
                    messagebox.showerror("Error", "Invalid destination.")
                    return
                move = None
                for m in self.valid_moves:
                    if m.dst == dst and (m.promotion == promotion or 
                                       (m.promotion is None and promotion is None)):
                        move = m
                        break
                if not move:
                    messagebox.showerror("Error", "Invalid move.")
                    return
                if isinstance(move.piece, Pawn) and (move.dst[0] in [0,7]) and not move.promotion:
                    messagebox.showerror("Error", "Promotion required (e.g. e8q).")
                    return
                self.board.move_piece(move)
                self.selected_src = None
                self.valid_moves = []
                self.dst_notations = []
                self.draw_board()
                self.switch_turn()

    def switch_turn(self):
        if self.board.in_checkmate(self.current):
            winner = 'AI' if self.current == self.human.color else 'Human'
            messagebox.showinfo("Game Over", f"Checkmate! {winner} wins!")
            self.root.quit()
            return
        if self.board.stalemate(self.current):
            messagebox.showinfo("Game Over", "Stalemate!")
            self.root.quit()
            return
        self.current = 'black' if self.current == 'white' else 'white'
        self.update_status()
        if self.current == self.ai.color:
            self.root.after(100, self.ai_move)

    def ai_move(self):
        move = self.ai.choose_move(self.board)
        if move:
            self.board.move_piece(move)
            self.draw_board()
        self.switch_turn()

if __name__ == '__main__':
    ChessGame()